# PUBG prediction 
https://www.kaggle.com/c/pubg-finish-placement-prediction/data

## Install Kaggle api 
https://github.com/Kaggle/kaggle-api

In [16]:
#! sudo /opt/anaconda3/bin/pip install kaggle --upgrade
#! mkdir -p ~/.kaggle/
# ! mv ~/kaggle.json ~/.kaggle/

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [8]:
from fastai import *
from fastai.tabular import * 
from tqdm import tqdm

In [12]:
path = Path(Config.get_key('data_path')).expanduser()/'pubg'
os.makedirs(path, exist_ok=True)
path

PosixPath('/home/jupyter/.fastai/data/pubg')

In [30]:
# ! /opt/anaconda3/bin/kaggle competitions download -c pubg-finish-placement-prediction -f train_V2.csv -p {path}
# ! /opt/anaconda3/bin/kaggle competitions download -c pubg-finish-placement-prediction -f test_V2.csv -p {path}
# ! unzip {path}/train_V2.csv.zip -d {path}
# ! unzip {path}/test_V2.csv.zip -d {path}
# !chmod 775 -R {path}

In [31]:
df = pd.read_csv((path/'train_V2.csv').as_posix())

In [33]:
df.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.00,0,0,0,60,...,0,0.0000,0,0.00,0,0,244.80,1,1466,0.4444
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.47,0,0,0,57,...,0,0.0045,0,11.04,0,0,1434.00,5,0,0.6400
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.00,0,0,0,47,...,0,0.0000,0,0.00,0,0,161.80,2,0,0.7755
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90,0,0,0,75,...,0,0.0000,0,0.00,0,0,202.70,3,0,0.1667
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,0,0,100.00,0,0,0,45,...,0,0.0000,0,0.00,0,0,49.75,2,0,0.1875


In [ ]:
df = df[df['maxPlace'] > 1]
df['totalDistance'] = df['rideDistance'] + df["walkDistance"] + df["swimDistance"]
print("remove some columns")
target = 'winPlacePerc'
features = list(df.columns)
features.remove("Id")
features.remove("matchId")
features.remove("groupId")

features.remove("matchType")
y = None
print("get target")
y = np.array(df.groupby(['matchId','groupId'])[target].agg('mean'), dtype=np.float64)
features.remove(target)
agg = df.groupby(['matchId','groupId'])[features].agg('mean')
agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
df_out = agg.reset_index()[['matchId','groupId']]
df_out = df_out.merge(agg.reset_index(), suffixes=["", ""], how='left', on=['matchId', 'groupId'])
df_out = df_out.merge(agg_rank, suffixes=["_mean", "_mean_rank"], how='left', on=['matchId', 'groupId'])
print("get group max feature")
agg = df.groupby(['matchId','groupId'])[features].agg('max')
agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
df_out = df_out.merge(agg.reset_index(), suffixes=["", ""], how='left', on=['matchId', 'groupId'])
df_out = df_out.merge(agg_rank, suffixes=["_max", "_max_rank"], how='left', on=['matchId', 'groupId'])

print("get group min feature")
agg = df.groupby(['matchId','groupId'])[features].agg('min')
agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
df_out = df_out.merge(agg.reset_index(), suffixes=["", ""], how='left', on=['matchId', 'groupId'])
df_out = df_out.merge(agg_rank, suffixes=["_min", "_min_rank"], how='left', on=['matchId', 'groupId'])

print("get group size feature")
agg = df.groupby(['matchId','groupId']).size().reset_index(name='group_size')
df_out = df_out.merge(agg, how='left', on=['matchId', 'groupId'])

print("get match mean feature")
agg = df.groupby(['matchId'])[features].agg('mean').reset_index()
df_out = df_out.merge(agg, suffixes=["", "_match_mean"], how='left', on=['matchId'])
print("get match size feature")
agg = df.groupby(['matchId']).size().reset_index(name='match_size')
df_out = df_out.merge(agg, how='left', on=['matchId'])

df_out.drop(["matchId", "groupId"], axis=1, inplace=True)

X = np.array(df_out, dtype=np.float64)

feature_names = list(df_out.columns)


remove some columns
get target
